### Load Data

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/Work/Aarete (Fall 2021)/Molina Letters'

/content/gdrive/MyDrive/Work/Aarete (Fall 2021)/Molina Letters


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
npi_info = pd.read_excel('NPI Info.xlsx', sheet_name='PDM Validation Detail Final', header=2).iloc[1:].reset_index(drop=True)
npi_info['NPI'] = npi_info['NPI'].astype(int)
npi_info.head()

,Owner,NPI,Validate,Update,Termination Date if Applicable,Last_Name/ Facility_Name,Validate.1,Update.1,First_Name,Validate.2,Update.2,Middle_ Name (only if populated),Validate.3,Update.3,Email,Validate.4,Update.4,License,Validate.5,Update.5,LicenseBoard,Validate.6,Update.6,Primary_ Specialty,Validate.7,Update.7,Certification_Board_Primary,Validate.8,Update.8,Secondary_Specialty1,Validate.9,Update.9,Certification_Board_Secondary1,Validate.10,Update.10,Secondary_Specialty2,Validate.11,Update.11,Certification_Board_Secondary2,Validate.12,...,Thursday,Unnamed: 87,Validate.27,Update.27,Friday,Unnamed: 91,Validate.28,Update.28,Saturday,Unnamed: 95,Validate.29,Update.29,Sunday,Unnamed: 99,Validate.30,Update.30,Accepting_New_ Patient_ Pgm,Validate.31,Update.31,Line of Business,Validate.32,Update.32,Affiliated Group 1,Validate.33,Update.33,Affiliated Group 2,Validate.34,Update.34,Affiliated Group 3,Validate.35,Update.35,Affiliated Group 4,Validate.36,Update.36,Affiliated Group 5,Validate.37,Update.37,Unnamed: 123,Final Outcome,Summary Comments
0,CC,1003830092,1. Affirmed,NaN,NaN,BUSHNELL,1. Affirmed,NaN,FREDERIC,1. Affirmed,NaN,R,1. Affirmed,NaN,NaN,3. New Info,rbushnell1231@yahoo.com,NaN,3. New Info,A74349,NaN,3. New Info,STATE BOARD OF MEDICAL EXAMINERS,Anesthesiology,1. Affirmed,NaN,AMERICAN BOARD OF ANESTHESIOLOGY INC.,1. Affirmed,NaN,NaN,1. Affirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,08:00:00,17:00:00,6. No Longer Valid,7:00 AM - 5:00 PM,08:00:00,17:00:00,6. No Longer Valid,7:00 AM - 5:00 PM,CLOSED,CLOSED,1. Affirmed,NaN,CLOSED,CLOSED,1. Affirmed,NaN,Y,1. Affirmed,NaN,Dual,1. Affirmed,NaN,EL PROYECTO DEL BARRIO INC,1. Affirmed,NaN,EL PROYECTO DEL BARRIO INC,1. Affirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2. Validated with Changes,"Doctor no longer sees patients at one address,..."
1,CC,1003830092,1. Affirmed,NaN,NaN,BUSHNELL,1. Affirmed,NaN,FREDERIC,1. Affirmed,NaN,R,1. Affirmed,NaN,NaN,3. New Info,rbushnell1231@yahoo.com,NaN,3. New Info,A74349,NaN,3. New Info,STATE BOARD OF MEDICAL EXAMINERS,Anesthesiology,1. Affirmed,NaN,AMERICAN BOARD OF ANESTHESIOLOGY INC.,1. Affirmed,NaN,NaN,1. Affirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,08:00:00,17:00:00,6. No Longer Valid,7:00 AM - 5:00 PM,08:00:00,17:00:00,6. No Longer Valid,7:00 AM - 5:00 PM,CLOSED,CLOSED,1. Affirmed,NaN,CLOSED,CLOSED,1. Affirmed,NaN,Y,1. Affirmed,NaN,Marketplace,1. Affirmed,NaN,EL PROYECTO DEL BARRIO INC,1. Affirmed,NaN,EL PROYECTO DEL BARRIO INC,1. Affirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2. Validated with Changes,"Doctor no longer sees patients at one address,..."
2,CC,1003830092,1. Affirmed,NaN,NaN,BUSHNELL,1. Affirmed,NaN,FREDERIC,1. Affirmed,NaN,R,1. Affirmed,NaN,NaN,3. New Info,rbushnell1231@yahoo.com,NaN,3. New Info,A74349,NaN,3. New Info,STATE BOARD OF MEDICAL EXAMINERS,Anesthesiology,1. Affirmed,NaN,AMERICAN BOARD OF ANESTHESIOLOGY INC.,1. Affirmed,NaN,NaN,1. Affirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,08:00:00,17:00:00,6. No Longer Valid,7:00 AM - 5:00 PM,08:00:00,17:00:00,6. No Longer Valid,7:00 AM - 5:00 PM,CLOSED,CLOSED,1. Affirmed,NaN,CLOSED,CLOSED,1. Affirmed,NaN,Y,1. Affirmed,NaN,Medicaid,1. Affirmed,NaN,EL PROYECTO DEL BARRIO INC,1. Affirmed,NaN,EL PROYECTO DEL BARRIO INC,1. Affirmed,NaN,SHRINERS FOR CHILDREN PROFESSIONAL SERVICES,1. Affirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2. Validated with Changes,"Doctor no longer sees patients at one address,..."
3,CC,1003830092,1. Affirmed,NaN,NaN,BUSHNELL,1. Affirmed,NaN,FREDERIC,1. Affirmed,NaN,R,1. Affirmed,NaN,NaN,3. New Info,rbushnell1231@yahoo.com,NaN,3. New Info,A74349,NaN,3. New Info,STATE BOARD OF MEDICAL EXAMINERS,Anesthesiology,1. Affirmed,NaN,AMERICAN BOARD OF ANESTHESIOLOGY INC.,1. Affirmed,NaN,NaN,1. Affirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,08:00:00,17:00:00,6. No Longer Valid,7:00 AM - 5:00 PM,08:00:00,17:00:00,6. No Longer Valid,7:00 AM - 5:00 PM,CLOSED,CLOSED,1. Affirmed,NaN,CLOSED,CLOSED,1. Affirmed,NaN,Y,1. Affirmed,NaN,Medicaid,1. Affirmed,NaN,SHRINERS F

In [ ]:
active_npi = pd.read_excel('NPI Info.xlsx', sheet_name='NPI Detail', header=2)
active_npi = active_npi[~(active_npi['Final Outcome'] == '7. Terminated')].reset_index(drop=True)
active_npi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   owner                     196 non-null    object        
 1   NPI                       284 non-null    int64         
 2   Last_Name/ Facility_Name  284 non-null    object        
 3   First_Name                282 non-null    object        
 4   Date First Submitted      88 non-null     datetime64[ns]
 5   Final Outcome             284 non-null    object        
 6   Summary Notes             281 non-null    object        
 7   Additional Notes          124 non-null    object        
 8   Column1                   0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 20.1+ KB


In [ ]:
letter_info_template = pd.read_excel('Table Format for Letters.xlsx', sheet_name='Sheet1', header=1)
letter_info_template.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 3 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Provider Data Fields                               55 non-null     object 
 1   Provider Data Information Based on Recent Updates  0 non-null      float64
 2   Provider Comments Noting Exceptions                0 non-null      float64
dtypes: float64(2), object(1)
memory usage: 1.4+ KB


In [ ]:
letter_info_template['Provider Data Fields'].unique()

array(['NPI', 'Last_Name/ Facility_Name', 'First_Name',
       'Middle_ Name (only if populated)', 'Email', 'License',
       'LicenseBoard', 'Primary_ Specialty',
       'Certification_Board_Primary', 'Secondary_Specialty1',
       'Certification_Board_Secondary1', 'Secondary_Specialty2',
       'Certification_Board_Secondary2', 'Secondary_Specialty3',
       'Certification_Board_Secondary3', 'Secondary_Specialty4',
       'Certification_Board_Secondary4', 'Address', 'City', 'State',
       'ZIP', 'Service_ Location_ County', 'Phone', 'Fax', 'Monday',
       'Hrs_To', 'Hrs_From', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
       'Saturday', 'Sunday', 'Accepting_New_ Patient_ Pgm', 'Medicad',
       'Medicare', 'Marketplace', 'Duals', 'Affiliated Group 1',
       'Affiliated Group 2', 'Affiliated Group 3', 'Affiliated Group 4',
       'Affiliated Group 5'], dtype=object)

### Remove Terminated NPIs

In [ ]:
active_npi = npi_info[npi_info['NPI'].isin(active_npi['NPI'])].reset_index(drop=True)
active_npi = active_npi.rename(columns={'Monday': 'Monday Hrs_From', 'Unnamed: 75': 'Monday Hrs_To', 
                           'Tuesday': 'Tuesday Hrs_From', 'Unnamed: 79': 'Tuesday Hrs_To', 
                           'Wednesday': 'Wednesday Hrs_From', 'Unnamed: 83': 'Wednesday Hrs_To', 
                           'Thursday': 'Thursday Hrs_From', 'Unnamed: 87': 'Thursday Hrs_To', 'Friday': 'Friday Hrs_From', 
                           'Unnamed: 91': 'Friday Hrs_To', 'Saturday': 'Saturday Hrs_From', 'Unnamed: 95': 'Saturday Hrs_To', 
                           'Sunday': 'Sunday Hrs_From', 'Unnamed: 99': 'Sunday Hrs_To'})
active_npi.shape

(2155, 126)

### Remove NPIs Without Addresses

In [ ]:
invalid_npi_addresses = active_npi[(active_npi['Validate.17'] != '1. Affirmed') & (active_npi['Update.17'].isna())].index
active_npi = active_npi.drop(invalid_npi_addresses).reset_index(drop=True)
active_npi.shape

(1678, 126)

### Transform 

In [ ]:
prov_feats = ['NPI', 'Last_Name/ Facility_Name', 'First_Name', 'Middle_ Name (only if populated)', 'Email', 'License', 
              'LicenseBoard', 'Primary_ Specialty', 'Certification_Board_Primary', 'Secondary_Specialty1', 'Secondary_Specialty2', 
              'Certification_Board_Secondary1', 'Certification_Board_Secondary2', 'Secondary_Specialty3', 
              'Certification_Board_Secondary3', 'Secondary_Specialty4', 'Certification_Board_Secondary4', 
             'Address', 'City', 'State', 'ZIP', 'Service_ Location_ County', 'Phone', 'Fax', 
              'Monday Hrs_From', 'Monday Hrs_To', 'Tuesday Hrs_From', 'Tuesday Hrs_To', 'Wednesday Hrs_From', 'Wednesday Hrs_To', 
              'Thursday Hrs_From', 'Thursday Hrs_To', 'Friday Hrs_From', 'Friday Hrs_To', 'Saturday Hrs_From', 'Saturday Hrs_To', 
              'Sunday Hrs_From', 'Sunday Hrs_To', 'Accepting_New_ Patient_ Pgm', 'Line of Business', 'Affiliated Group 1', 'Affiliated Group 2', 
              'Affiliated Group 3', 'Affiliated Group 4', 'Affiliated Group 5']

table_to_transf = active_npi[prov_feats]
table_to_transf['ZIP'] = table_to_transf['ZIP'].astype(int)
table_to_transf.shape


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


(1678, 45)

In [ ]:
def parse_time(text, fmts=['%I:%M%p', '%I:%M %p', '%H', '%H:%M', '%I%p', '%I %p']):
    for fmt in fmts:
        try:
            return datetime.strptime(text, fmt).time()
        except ValueError:
            pass
    return text

In [ ]:
for i, row in table_to_transf.iterrows():
    for feat in prov_feats: 
        if 'Hrs_From' in feat:  
            if active_npi.iloc[i, (active_npi.columns.tolist().index(feat) + 2)] != '1. Affirmed': # if validate value of feature not affired
                hrs_updt = active_npi.iloc[i, (active_npi.columns.tolist().index(feat) + 3)]
                try:
                    hrs_from = parse_time(hrs_updt.split('-')[0].strip().upper())
                except AttributeError: # if nan
                    hrs_from = hrs_updt
                table_to_transf.loc[i, feat] = hrs_from # update feature value with update value
                
        elif 'Hrs_To' in feat:
            if active_npi.iloc[i, (active_npi.columns.tolist().index(feat) + 1)] != '1. Affirmed': # if validate value of feature not affired
                hrs_updt = active_npi.iloc[i, (active_npi.columns.tolist().index(feat) + 2)]
                try:
                    hrs_to = parse_time(hrs_updt.split('-')[1].strip().upper())
                except IndexError: # if CLOSED
                    hrs_to = hrs_updt
                except AttributeError:
                    hrs_to = hrs_updt # if nan
                    
                table_to_transf.loc[i, feat] = hrs_to # update feature value with update value
            
        else:
            if active_npi.iloc[i, (active_npi.columns.tolist().index(feat) + 1)] != '1. Affirmed': # if validate value of feature not affired
                table_to_transf.loc[i, feat] = active_npi.iloc[i, (active_npi.columns.tolist().index(feat) + 2)] # update feature value with update value

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
for group_num, group in table_to_transf.groupby('NPI'):
    aff_groups = pd.concat([group['Affiliated Group 1'], group['Affiliated Group 2'], group['Affiliated Group 3'], group['Affiliated Group 4'], group['Affiliated Group 5']]).dropna().unique()
    table_to_transf.loc[group.index, 'Affiliated Groups'] = '; '.join(aff_groups)

table_to_transf = table_to_transf.drop(columns=[f'Affiliated Group {i}' for i in range(1, 6)])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
table_for_letter_cols = ['NPI', 'Last_Name/ Facility_Name', 'First_Name',
       'Middle_ Name (only if populated)', 'Email', 'License', 'LicenseBoard',
       'Primary_ Specialty', 'Certification_Board_Primary',
       'Secondary_Specialty1', 'Secondary_Specialty2', 'Secondary_Specialty3', 
       'Secondary_Specialty4', 'Certification_Board_Secondary1',
       'Certification_Board_Secondary2','Certification_Board_Secondary3',
       'Certification_Board_Secondary4', 'Address', 'City', 'State', 'ZIP',
       'Service_ Location_ County', 'Phone', 'Fax', 'Monday Hrs_From',
       'Monday Hrs_To', 'Tuesday Hrs_From', 'Tuesday Hrs_To',
       'Wednesday Hrs_From', 'Wednesday Hrs_To', 'Thursday Hrs_From',
       'Thursday Hrs_To', 'Friday Hrs_From', 'Friday Hrs_To',
       'Saturday Hrs_From', 'Saturday Hrs_To', 'Sunday Hrs_From',
       'Sunday Hrs_To', 'Accepting_New_ Patient_ Pgm', 'Medicaid', 'Medicare',
       'Marketplace', 'Dual', 'Affiliated Groups']
tables_for_letter = []

for group_num, group in table_to_transf.groupby(['NPI', 'Address']):
    table_for_letter = pd.DataFrame(index=table_for_letter_cols, columns=['Provider Data Information Based on Recent Updates', 'Provider Comments Noting Exceptions'])
    for info_feat in group.columns:
        if info_feat == 'Line of Business':
            for lob in group['Line of Business'].values:
                table_for_letter.loc[lob, 'Provider Data Information Based on Recent Updates'] = 'Participating'
            for lob in ['Medicaid', 'Medicare','Marketplace', 'Dual']:
                if table_for_letter.loc[lob, 'Provider Data Information Based on Recent Updates'] != 'Participating':
                    table_for_letter.loc[lob, 'Provider Data Information Based on Recent Updates'] = 'Non-Participating'
        else:
            table_for_letter.loc[info_feat, 'Provider Data Information Based on Recent Updates'] = group[info_feat].values[0] 
    #table_for_letter = table_for_letter.iloc[:51] # ensure number of pages in letter < 4
    tables_for_letter.append(table_for_letter.reset_index().rename(columns={'index': 'Provider Data Fields'}).fillna(''))

**Save**

In [ ]:
import pickle

In [ ]:
with open('tables_for_letter.pkl', 'wb') as f:
    pickle.dump(tables_for_letter, f)